In [1]:
print('Hi')

Hi


In [5]:
import redis
redis_host = 'roundhouse.proxy.rlwy.net'
redis_port = 15902  # Default Redis port
redis_username = 'default'
redis_password = 'jlaagI6MiGIldHJADB524MAjDjKM1652'
redis_db = 0  # Default Redis database
def connect_to_redis(host, port, username, password, db=0):
    try:
        # Create a connection to the Redis server
        redis_client = redis.StrictRedis(
            host=host,
            port=port,
            username=username,
            password=password,
            db=db,
            decode_responses=True  # Decode responses to strings
        )
        # Test the connection
        redis_client.ping()
        print("Connected to Redis server successfully!")
        return redis_client
    except Exception as e:
        print(f"Failed to connect to Redis server: {e}")
        return None

redis_client = connect_to_redis(redis_host, redis_port, redis_username, redis_password)
redis_client.set("name", '{"name": "sam"}')
redis_client.get("name")

Connected to Redis server successfully!


'{"name": "sam"}'

In [20]:


from abc import ABC, abstractmethod
import json
from fastapi.encoders import jsonable_encoder
from pydantic import BaseModel
import redis

class ICache(ABC):
    @abstractmethod
    async def set_item(key: str, value: object) -> None:
        pass

    @abstractmethod
    async def get_item(key: str) -> object | None:
        pass


class Cache(ICache):
    def __init__(self, host, port, username, password, db=0):
        try:
            # Create a connection to the Redis server
            redis_client = redis.StrictRedis(
                host=host,
                port=port,
                username=username,
                password=password,
                db=db,
                decode_responses=True  # Decode responses to strings
            )
            # Test the connection
            redis_client.ping()
            self.redis_client = redis_client
        except Exception as e:
            self.redis_client = None
        
    async def set_item(self, key: str, value: object) -> None:
        if self.redis_client is None:
            return None
        json_value = json.dumps(jsonable_encoder(value))
        self.redis_client.set(key, json_value)

    async def get_item(self, key: str) -> object | None:
        if self.redis_client is not None:
            value = self.redis_client.get(key)
            if value:
                return json.loads(value)
        return None
    
class HumanAgeApiResponse(BaseModel):
    name: str
    age: int | None
    
class IHumanAgeCache(ABC):
    @abstractmethod
    async def get_age_for_name(self, name: str) -> HumanAgeApiResponse | None:
        pass
    
    @abstractmethod
    async def save_age_for_name(self, age_for_name: HumanAgeApiResponse) -> None:
        pass

class HumanAgeCache(IHumanAgeCache):
    def __init__ (self, cache: ICache):
        self.__cache = cache

    async def get_age_for_name(self, name: str) -> HumanAgeApiResponse | None:
        saved_age_for_name = await self.__cache.get_item(name)
        if saved_age_for_name is not None:
            return HumanAgeApiResponse(**saved_age_for_name)
        
    async def save_age_for_name(self, age_for_name: HumanAgeApiResponse) -> None:
        await self.__cache.set_item(age_for_name.name, age_for_name)

human_age_cache = HumanAgeCache()


In [21]:
await human_age_cache.save_age_for_name(HumanAgeApiResponse(name = "sam", age = 12))

In [23]:
await human_age_cache.get_age_for_name("sammy")

In [34]:
import aiohttp
import requests
class BaseApiRequest():
    async def get_data(self, url: str) -> str | None:
        response = requests.get(url)
        if response.ok:
            return response.json()
        else:
            return None
class IAgeApi(ABC):
    @abstractmethod
    async def get_age_by_name(self, name: str) -> HumanAgeApiResponse | None:
        pass
class AgeApi(BaseApiRequest, IAgeApi):

    async def get_age_by_name(self, name: str) -> HumanAgeApiResponse | None:
        request_url = f"https://api.agify.io/?name={name}"
        json_data = await self.get_data(request_url)
        if json_data is not None:
            
            age_response =  HumanAgeApiResponse(**json_data)
            return None if age_response.age is None else age_response
        
        return None
    
age_api = AgeApi()
await age_api.get_age_by_name("micheal")

HumanAgeApiResponse(name='micheal', age=57)